In [1]:
from dataset import *
import wandb
import torchvision.transforms as transforms
import torch.optim as optim
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor, FasterRCNN
from torchvision.models.detection.backbone_utils import resnet_fpn_backbone
import pandas as pd
import numpy as np

## Hyperparameters

In [175]:
### HYPERPARAM

batchsize = 2
in_dim = (300,300)
normalization_data = torch.load('mean-std.pt')
num_classes = 7

### CONSTANTS
diseases = ['akiec', 'bcc', 'bkl', 'df', 'mel', 'nv', 'vasc']
mapping = {'akiec': 0, 'bcc': 1, 'bkl': 2, 'df': 3, 'mel': 4, 'nv': 5, 'vasc': 6}
inv_mapping = {mapping[k]:k for k in mapping.keys()}


## Data Loading

In [171]:
dataset = SkinData('/', 'final.csv', transform=transforms.Compose([ToTensor, Normalizer(normalization_data)]))
train_data, test_data, valid_data = torch.utils.data.random_split(dataset,[int(0.7 * len(dataset)), int(0.15 * len(dataset)), int(0.15 * len(dataset))+1],  generator=torch.Generator().manual_seed(42))
data_loader_train = torch.utils.data.DataLoader(train_data, batch_size=batchsize, collate_fn = collate_fn)
data_loader_test = torch.utils.data.DataLoader(test_data, batch_size=batchsize, collate_fn = collate_fn)
data_loader_valid =  torch.utils.data.DataLoader(valid_data, batch_size=batchsize, collate_fn = collate_fn)

## Model

In [162]:
def initialize_model():
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes + 1)
    return model

In [163]:
model = initialize_model()

In [164]:
torch.save(model.state_dict(), 'fast_rcnn_v1_state.pt')
torch.save(model, 'fast_rcnn_v1.pt')

## Training

In [167]:
# Best Metric
best_val = 0
for epoch in range(config.epochs):
    for i, d in enumerate(data_loader_train):

        # Header
        header = 10*"="
        print(header, "Epoch {}".format(epoch), header)

        # Metric Initialization
        metrics = {name: ClassMeter(disease) for disease in diseases}
        metrics['train_loss'] = AverageMeter()
        metrics['val_loss'] = AverageMeter()
        metrics['val_mAP'] = AverageMeter()
        

        # Training
        model.train()

        image, target = d
        image = image.to(device)
        target = target.to(device)
        losses = model(image, target)
        loss = sum(loss for loss in losses.values())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        lr_scheduler.step()

        metrics['train_loss'].update(loss.item(), n=len(target))

        # Log to weights and biases
        # FIXME

        # Print every 10 epochs
        # FIXME

    with torch.no_grad():
        for i, d in enumerate(data_loader_valid):
            model.train()
            image, target = d
            image = image.to(device)
            losses = model(image)
            loss = sum(loss for loss in losses.values())
            metrics['val_loss'].update(loss.item(), n=len(target))
            
            model.eval()
            # Calculate mAP
            # FIXME
        
        # Print Summary
        # FIXME

        # Log Values
        # FIXME

        # Log bounding box image sample to WandB
        # FIXME

        # SAVE BEST MODEL
        # FIXME

        # SAVE EVERY FIVE EPOCHS
        # FIXME



        

({'area': 150997.0, 'labels': tensor([5]), 'boxes': tensor([[116.,  42., 487., 449.]], dtype=torch.float64), 'image_id': 'ISIC_0030405'}, {'area': 99084.0, 'labels': tensor([5]), 'boxes': tensor([[187.,  30., 463., 389.]], dtype=torch.float64), 'image_id': 'ISIC_0026515'})
{'loss_classifier': tensor(1.3107, grad_fn=<NllLossBackward>), 'loss_box_reg': tensor(0.1040, grad_fn=<DivBackward0>), 'loss_objectness': tensor(0.4460, grad_fn=<BinaryCrossEntropyWithLogitsBackward>), 'loss_rpn_box_reg': tensor(0.0337, dtype=torch.float64, grad_fn=<DivBackward0>)}
